images - line by line, cmaera - col by col
    Q: is this hardware specific?
screen waits 1 frame cycle, camera waits 1 frame cycle 
    Q: r they the same?


two kinds of challenges:
    background challenge: one color
    lighting challenge: belt of different colr from background color
        belt is "lighting area"

ROI: region that camera is scanning when the screen is displaying the lighting area

In [ ]:
import numpy as np

In [ ]:
def eqn_2(pixel, color1, color2, E):
    #E{r,g,b} = illuminant
    # r,g,b = pixel
    #confirm that I{c1}/I{c2} = E{c1}/E{c2}
    #(where c1 and c2 are the 2 colors being shown on the screen)  

    iFraction = pixel[color1]/pixel[color2]
    eFraction = E[color1]/E[color2]
    # ?? how to find the illuminant of a color being shown on the  screen ??
    epsilon = 0.5
    return iFraction- eFraction <= epsilon  


def roi(frames, t_begin, u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E):
    # STEP 1: Calculate ROI
    # calculate start time to show the lighting area, t_u
    t_u = t_begin + (u / rows) * t_frame
    # find image - frame at t_u
    image = frames[fps * t_u]
    # calculate shift, l
    l = cols * (t_u - ct_k) / (ct_frame)
    

    # Apply Eqn 2 on every pixel between response of lighting challenge and background challenge
    for pixel in image:
        consistent = eqn_2(pixel, color1, color2, E)
        if not consistent:
            print("Not consistent!")
            return
    a,b=10,50
    ROI = (a,b)
    # regression model
    #image is 100 columns, ROI is columns 40-65
    
    y_hat = 0
    if a < 0.25*imgWidth:
        #take average value of columns [a,25]
        #run regression on model 1
        y_hat += (25-a)*output
    if b < 0.5*imgWidth:
        #model2: avg. value of columns [25, b]
        # run regression on model 2 
        y_hat += (b-25)*output

    if b < 0.75*imgWidth and a > 0.25*imgWidth:
        #model 3: avg value of columns [50,b]
        # run regression on model 3
        y_hat += (b-50)*output

    if b >=0.75*imgWidth:
        #model 4 : avg value of columns [75,b]
        # run regression on model 4 
        y_hat += (b-75)*output
    
    y_hat = y_hat / (25)

    return y_hat


def criteria(frames):
    t_begin, u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E = 1,2,3,4,5,6,7,8,9,10,11
    
    dVals = []
    for frame in frames:
        y_hat_i = roi(frames, t_begin, u, rows, t_frame, cols, ct_k, ct_frame, fps, color1, color2, E)

        # calc criteria
        d = 0.25
        imgRows = 
        
        d_i = y_hat_i - (u + imgRows*0.25)/2
        dVals.append(d_i)

    mean = np.mean(dVals)
    var = np.var(dVals)

    threshold = -5
    return mean * np.sqrt(var) < np.exp(threshold)
